In [1]:
import requests
import re
import time
import urllib
import os

from urllib.request import urlretrieve
from bs4 import BeautifulSoup
from urllib.error import HTTPError
from urllib.error import URLError
from bs4 import BeautifulSoup
from progressbar import *


In [ ]:
# 修正了只能下載一頁的問題，但偶爾還是會被檔爬蟲，所以謹慎使用
# 使用間隔別太短，載第二本請等待至少3分鐘

# 直接執行以下code

In [3]:
class browser:
    def download():       
        url = input("please enter URL:")

        headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) \
            AppleWebKit/537.36 (KHTML, like Gecko) \
                Chrome/35.0.1916.114 Safari/537.36',
            'Cookie': 'AspxAutoDetectCookieSupport=1'}

        response = requests.get(url,headers = headers) #使用header避免訪問受到限制
        req = urllib.request.Request(url=url, headers=headers)
        html = urllib.request.urlopen(req).read().decode('utf8')
        soup = BeautifulSoup(html, 'html.parser') # 使用BeautifulSoup整理html原始碼


        # #觀看共有幾頁，確定需要幾個迴圈
        # len(soup.find_all('div',{'class':'gdtm'}))
        # total = len(soup.find_all('div',{'class':'gdtm'}))  #存入total等等計數用


        html_page = soup.find_all('table',{'class':'ptt'})  #用來解析共有幾頁
        how_page = []  #存入幾頁用
        for i in html_page:
            how_page += (i.text)

        how_page2 = ( len (how_page)-2 )   
        #去掉頭尾< > 符號，真實頁數，也就是需要抓幾頁



        #這邊自動命名資料夾名稱，如果沒有漢化的就用原本的英文命名
        folder_name =[]
        if bool( soup.find_all('h1')[1].text) == True:
            folder_name = soup.find_all('h1')[1].text
            # strip = 移除頭尾空白
        else:
            folder_name = soup.find_all('h1')[0].text


        # 設定資料夾位置
        folder_path='C:/' +folder_name

        if os.path.exists(folder_path) == False:  # 判斷資料夾是否存在
            os.makedirs(folder_path)  # 不存在則創一個       

        # 使用opener 

        opener=urllib.request.build_opener()

        header_list = []

        #將headers裡面的物件放入header_list
        for key, value in headers.items():
            header_list.append((key, value))

        opener.addheaders = header_list 
        urllib.request.install_opener(opener)


        link = []  #存連結用
        count=1 #顯示進度用
        total_page = 0  #存取實際幾頁漫畫用

        ###################################計算頁數用####################################
        for i in range(how_page2):
        #總共有幾頁 實際的url迴圈數    
            url2 = url+'?p='+str(i)   
            response = requests.get(url2,headers = headers) #使用header避免訪問受到限制
            req = urllib.request.Request(url=url2, headers=headers)
            html = urllib.request.urlopen(req).read().decode('utf8')
            soup = BeautifulSoup(html, 'html.parser') # 使用BeautifulSoup整理html原始碼
            for j in range(len(soup.find_all('div',{'class':'gdtm'}))):
                total_page+=1
        #################################################################################


        progress = ProgressBar() #顯示進度條
        pbar = ProgressBar().start() #顯示進度條

        for i in range(how_page2):
        #總共有幾頁 實際的url迴圈數    
            url2 = url+'?p='+str(i)   
            response = requests.get(url2,headers = headers) #使用header避免訪問受到限制
            req = urllib.request.Request(url=url2, headers=headers)
            html = urllib.request.urlopen(req).read().decode('utf8')
            soup = BeautifulSoup(html, 'html.parser') # 使用BeautifulSoup整理html原始碼


            html2=[]
            for j in range(len(soup.find_all('div',{'class':'gdtm'}))):
                html2 += soup.find_all('div',{'class':'gdtm'})[j].find_all('a')      
                #尋找div標籤下 class的 gdtm
                #不過gdtm是寫死的 估計ehentai之後改版會將這個改掉，這個爬蟲就失效了

            for k in html2:
                url3 = k.get('href')  #抓到每一頁的漫畫 存進url3
                req = urllib.request.Request(url=url3, headers=headers)
                html = urllib.request.urlopen(req).read().decode('utf8')
                soup = BeautifulSoup(html, 'html.parser') #更新soup
                link.append(soup.find_all('img')[4].get('src'))   #陣列[4]寫死的是因為是圖片原始網址
                pbar.update( ( count / total_page )*100 )  #顯示進度條
                count+=1
                time.sleep(0.5)

        pbar.finish()


        # 開始把link陣列裡的圖片抓下來
        x=1 #顯示進度用

        progress = ProgressBar() #顯示進度條
        pbar = ProgressBar().start() #顯示進度條

        for i in range(len(link)):
            local = os.path.join( folder_path + '\ %s.jpg' % x)
            urllib.request.urlretrieve(link[i] ,local) #link是下載的網址 local是儲存圖片的檔案位址
            pbar.update( ( x / total_page )*100 )  #顯示進度條
            time.sleep(1) #睡眠一下避免被檔
            x+=1
        pbar.finish()
        input("press enter to exit...")

# 執行以下輸入網址

In [4]:
# 雖然依然會有被擋爬蟲的問題

# 共兩條進度條
# 1為作業進度，2為下載進度
browser.download()

please enter URL:https://e-hentai.org/g/1362311/be0eca4897/


100% |########################################################################|
100% |########################################################################|


press enter to exit...


In [2]:
# https://e-hentai.org/g/1362311/be0eca4897/

# https://e-hentai.org/g/1362261/5762725d12/

# https://e-hentai.org/g/1327890/eb5a229cdd/